In [20]:
# import Libraries
import os
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone

from langchain_google_genai import GoogleGenerativeAI,GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [31]:
llm=GoogleGenerativeAI(model="gemini-pro", temperature=0.5,api_key=GOOGLE_API_KEY)
embedding = GoogleGenerativeAIEmbeddings(model = "models/embedding-001",api_key=GOOGLE_API_KEY)

In [25]:
from langchain.text_splitter import CharacterTextSplitter
from pypdf import PdfReader
import glob

class readpdf:
    def __init__(self, pdf_docs) :
        self.pdf_docs=pdf_docs
        self.text = ""

    def get_pdf_text(self):
        for pdf in self.pdf_docs:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                content=page.extract_text() 
                if content:
                    self.text += content 

    def get_text_chunks(self):
        text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=50,
        length_function=len
        )
        chunks = text_splitter.split_text(self.text)
        return chunks
    
pdf_files = glob.glob("*.pdf")
text = readpdf(pdf_files)
text.get_pdf_text()
chunks=text.get_text_chunks()


In [26]:
len(chunks)

118

In [29]:
import cassio
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

ASTRA_DB_ID=os.getenv("ASTRA_DB_ID")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")

cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)
astra_vector_store=Cassandra(
    embedding=embedding,
    table_name="test",
    session=None,
    keyspace=None,
)

astra_vector_store.add_texts(chunks)
print("Inserted %i headlines." % len(chunks))
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 118 headlines.


In [35]:
query_text="How much the agriculture target will be increased by how many percentage?"
answer=astra_vector_index.query(query_text,llm=llm).strip()
print(answer)

I don't know. The provided context does not mention anything about the percentage increase in the agriculture target.
